In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [78]:
order_items=pd.read_csv('/Users/jiin/dacon/KPI_Dacon_2024/data/order_items.csv')
orders=pd.read_csv('/Users/jiin/dacon/KPI_Dacon_2024/data/orders.csv')

In [79]:
#주문상태가 delivered 인 주문만 
orders=orders[orders['Order_status']=='delivered']

In [22]:
orders.head()

,Order_id,Customer_id,Order_status,Order_purchase_timestamp,Order_delivered_carrier_date,Order_delivered_customer_date,Order_estimated_delivery_date
0,ORDER_00000,CUSTOMER_62232,delivered,2018-10-02 10:56:33,2018-10-04 19:55:00,2018-10-10 21:25:13,2018-10-18
1,ORDER_00001,CUSTOMER_54059,delivered,2018-11-18 19:28:06,2018-11-22 13:39:59,2018-12-02 00:28:42,2018-12-15
2,ORDER_00002,CUSTOMER_59535,delivered,2019-02-13 21:18:39,2019-02-14 19:46:34,2019-02-16 18:17:02,2019-02-26
3,ORDER_00003,CUSTOMER_60516,delivered,2018-07-09 21:57:05,2018-07-11 14:58:04,2018-07-26 10:57:55,2018-08-01
4,ORDER_00004,CUSTOMER_76957,delivered,2018-05-16 13:10:30,2018-05-22 10:07:46,2018-05-26 12:55:51,2018-06-07


In [23]:
order_items.head()

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value
0,ORDER_75477,1,PRODUCT_23128,SELLER_0452,58.9,13.29
1,ORDER_63604,1,PRODUCT_24351,SELLER_0414,239.9,19.93
2,ORDER_05566,1,PRODUCT_20226,SELLER_1637,199.0,17.87
3,ORDER_04632,1,PRODUCT_07878,SELLER_1445,199.9,18.14
4,ORDER_61452,1,PRODUCT_03505,SELLER_0581,21.9,12.69


In [80]:
orders['Order_purchase_timestamp']=pd.to_datetime(orders['Order_purchase_timestamp'])

## RFM 생성
- Recency (R): 가장 최근 구매로부터 경과한 시간
- Frequency (F): 총 구매 횟수
- Monetary (M): 총 구매 금액

여기서 궁금한 점. Frequency가 한번에 살때 20개 산것도 구매횟수로 치는건지 아니면 몇개를 사던 상관없이 한번 주문했을 때를 횟수는 치는 건지 궁금.


In [50]:
#주문별 총 구매 금액 계산
order_items['Total']=order_items['Price']+order_items['Freight_value']
order_total=order_items.groupby('Order_id')['Total'].sum().reset_index()

#주문 데이터와 주문 총액 데이터 병합
orders=pd.merge(orders,order_total,on='Order_id')

#Frequency
frequency=orders.groupby('Customer_id').size().reset_index(name='Frequency')

#Monetary
monetary=orders.groupby('Customer_id')['Total'].sum().reset_index()

#Recency
current_date=orders['Order_purchase_timestamp'].max()+timedelta(days=1)
orders['Recency']=(current_date-pd.to_datetime(orders['Order_purchase_timestamp'])).dt.days
recency=orders.groupby('Customer_id')['Recency'].min().reset_index()

rfm=pd.merge(recency,frequency,on='Customer_id')
rfm=pd.merge(rfm,monetary,on='Customer_id')

In [51]:
rfm

,Customer_id,Recency,Frequency,Total
0,CUSTOMER_00000,440,1,146.87
1,CUSTOMER_00001,199,1,335.48
2,CUSTOMER_00002,72,1,157.73
3,CUSTOMER_00003,139,1,173.30
4,CUSTOMER_00004,319,1,282.21
...,...,...,...,...
87943,CUSTOMER_88083,114,1,88.78
87944,CUSTOMER_88084,118,1,129.06
87945,CUSTOMER_88085,113,1,56.04
87946,CUSTOMER_88086,269,1,711.07


## RFM score

In [57]:
#R,F,M 값에 대해 4개의 구간으로 나누어 점수 부여
rfm['R']=pd.qcut(rfm['Recency'],5,labels=[5,4,3,2,1])
rfm['F']=pd.qcut(rfm['Frequency'].rank(method='first'),5,labels=[1,2,3,4,5])
rfm['M']=pd.qcut(rfm['Total'],5,labels=[1,2,3,4,5])

#계산
rfm['RFM_score']=rfm['R'].astype(str)+rfm['F'].astype(str)+rfm['M'].astype(str)

In [58]:
rfm

,Customer_id,Recency,Frequency,Total,R,F,M,RFM_score
0,CUSTOMER_00000,440,1,146.87,1,1,4,114
1,CUSTOMER_00001,199,1,335.48,3,1,5,315
2,CUSTOMER_00002,72,1,157.73,5,1,4,514
3,CUSTOMER_00003,139,1,173.30,4,1,4,414
4,CUSTOMER_00004,319,1,282.21,2,1,5,215
...,...,...,...,...,...,...,...,...
87943,CUSTOMER_88083,114,1,88.78,4,5,3,453
87944,CUSTOMER_88084,118,1,129.06,4,5,4,454
87945,CUSTOMER_88085,113,1,56.04,4,5,2,452
87946,CUSTOMER_88086,269,1,711.07,2,5,5,255


In [59]:
rfm.to_csv('RFM.csv',index=False)

In [61]:
#R
rfm.groupby('R').count()['Customer_id']

/var/folders/lf/x34jc_5j1gj6l3l68pp7fwj80000gn/T/ipykernel_30808/3463822983.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  rfm.groupby('R').count()['Customer_id']


R
5    17592
4    17742
3    18019
2    17123
1    17472
Name: Customer_id, dtype: int64

In [63]:
#F
rfm.groupby('F').count()['Customer_id']

/var/folders/lf/x34jc_5j1gj6l3l68pp7fwj80000gn/T/ipykernel_30808/3337474056.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  rfm.groupby('F').count()['Customer_id']


F
1    17590
2    17589
3    17590
4    17589
5    17590
Name: Customer_id, dtype: int64

In [68]:
#
rfm.groupby(['M']).agg({'Customer_id':'count','Total':'sum'})

/var/folders/lf/x34jc_5j1gj6l3l68pp7fwj80000gn/T/ipykernel_30808/1534917482.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  rfm.groupby(['M']).agg({'Customer_id':'count','Total':'sum'})


,Customer_id,Total
M,,
1,17590,687055.34
2,17591,1212090.91
3,17593,1855820.70
4,17584,2815230.01
5,17590,7465505.62


## RFM 시각화

R-F 행렬을 만들자, RFM 동시에 시각화하면 3차원이라서 가독성이 떨어진다. 

In [76]:
rf_matrix=rfm.groupby(['R','F']).agg({'Customer_id':'count','Total':'sum'}).reset_index()
rf_matrix['avg_sales']=rf_matrix['Total']/rf_matrix['Customer_id']
rf_matrix

/var/folders/lf/x34jc_5j1gj6l3l68pp7fwj80000gn/T/ipykernel_30808/48781263.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  rf_matrix=rfm.groupby(['R','F']).agg({'Customer_id':'count','Total':'sum'}).reset_index()


,R,F,Customer_id,Total,avg_sales
0,5,1,3522,569827.94,161.791011
1,5,2,3534,563604.79,159.480699
2,5,3,3513,588471.01,167.512385
3,5,4,3517,601292.38,170.967410
4,5,5,3506,572457.12,163.279270
5,4,1,3615,567222.77,156.908097
6,4,2,3479,557491.02,160.244616
7,4,3,3501,564306.73,161.184442
8,4,4,3606,563526.75,156.274750
9,4,5,3541,566608.50,160.013697


In [73]:
rf_pivot=rf_matrix.pivot(index='F',columns='R',values='Customer_id').fillna(0).sort_index(ascending=False)
rf_pivot.style.background_gradient(cmap='Greens',axis=None,low=0.2)

R,5,4,3,2,1
F,,,,,
5,3506,3541,3649,3386,3508
4,3517,3606,3554,3396,3516
3,3513,3501,3663,3513,3400
2,3534,3479,3623,3432,3521
1,3522,3615,3530,3396,3527


R-F 등급별로 속하는 사용자 수. 3-3이 가장 많다. 

In [77]:
#rf 등급별 평균 구매 금액
rf_pivot=rf_matrix.pivot(index='F',columns='R',values='avg_sales').fillna(0).sort_index(ascending=False)
rf_pivot.style.background_gradient(cmap='Greens',axis=None,low=0.2)

R,5,4,3,2,1
F,,,,,
5,163.279270,160.013697,157.234190,165.162094,162.435744
4,170.967410,156.274750,151.510248,164.184226,154.353308
3,167.512385,161.184442,155.979427,160.624677,155.531929
2,159.480699,160.244616,149.195664,163.054548,156.622664
1,161.791011,156.908097,143.194017,167.153716,167.328381


- R=5,F=5 : 충성고객, 이탈하지 않도록 관리하자
- R=5,F=4 : 지출 금액이 가장 큼, 최근에 구매했고, 구매횟수도 꽤 많은 고객. 충성고객으로 만들기 위해 노력해야함
- R=5,F=3 : 지출금액이 그다음으로 큼. 더 자주 구매할 수 있도록 고객 맞춤 마케팅 적극적으로 하자!!!
- R=2,F=1 & R=1,F=1: 오래전에 구매했고 거의 몇번 구매 안한 고객. 근데 지출이 크다. 왜지. 아마 신규고객유치를 위한 특별 이벤트나 프로모션을 진행했을 때 구매했던 고객들인 듯. 다시 재방문할 수 있도록 재방문 시 파격적인 이벤트를 열거나 쿠폰을 뿌리자!!